In [1]:
import re
import sys
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
sys.path.append('../..')

In [3]:
from ingestion.ingestion_mediator import IngestionMediator
mediator = IngestionMediator()

/home/johnny/jk-apps/bike-price-predictor/env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
# Get current raw data files for source
SOURCE = 'backcountry'
def get_csv_file_path(specs=True):
    if specs:
        tablename = 'product_specs'
    else:
        tablename = 'products'
    manifest_row = mediator.get_rows_matching(sources=[SOURCE], tablenames=[tablename])[0]
    return mediator.get_filepath_for_manifest_row(row=manifest_row)

csv_prods = get_csv_file_path(specs=False)
print('Prods csv path:', csv_prods)
csv_specs = get_csv_file_path()
print('Specs csv path:', csv_specs)

prods_df = pd.read_csv(csv_prods)
specs_df = pd.read_csv(csv_specs)

Prods csv path: /home/johnny/jk-apps/bike-price-predictor/data/raw_data/01112020/backcountry_prods_all.csv
Specs csv path: /home/johnny/jk-apps/bike-price-predictor/data/raw_data/01112020/backcountry_specs_all.csv


In [5]:
# Confirm have at least 'product_id' column to merge prods_df on specs_df
print('Same number of rows:', len(prods_df) == len(specs_df))
common_cols = set(prods_df.columns) & set(specs_df.columns)
print('Intersecting columns:', common_cols)

# Confirm same number of product ids - symmetric difference i.e. unique to each should be zero
set(prods_df.product_id.unique()) ^ set(specs_df.product_id.unique())

Same number of rows: True
Intersecting columns: {'product_id', 'site'}


set()

In [6]:
# Merge on common columns
merged_df = prods_df.merge(right=specs_df, how='outer', on=list(common_cols))
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403 entries, 0 to 402
Data columns (total 75 columns):
site                      403 non-null object
bike_type                 403 non-null object
subtype                   403 non-null object
product_id                403 non-null object
href                      403 non-null object
brand                     403 non-null object
description               403 non-null object
price                     403 non-null float64
msrp                      403 non-null float64
tires                     330 non-null object
bar_tape                  46 non-null object
battery_type              6 non-null object
computer                  4 non-null object
handlebar_drop            36 non-null object
brakeset                  331 non-null object
claimed_weight            80 non-null object
iscg_tabs                 233 non-null object
pedals                    202 non-null object
crank_arm_length          208 non-null object
details                   

In [8]:
# let's look at bike_type vs subtypes
grouped = merged_df.groupby(by=['bike_type', 'subtype'])
grouped.count()

site  product_id  \
bike_type               subtype                                              
ebikes                  emountain_bikes                      2           2   
                        eroad_bikes                          3           3   
                        eutility_bikes                       1           1   
gravel_cyclocross_bikes complete_gravel_cyclocross_bikes    47          47   
mountain_bikes          downhill_bikes                       3           3   
                        enduro_full_suspension_bikes        81          81   
                        hardtail_bikes                      25          25   
                        trail_full_suspension_bikes        146         146   
                        xc_full_suspension_bikes            37          37   
road_bikes              complete_road_bikes                 54          54   
triathlon_tt_bikes      complete_triathlon_tt_bikes          4           4   

                                                          href  brand  \
bike_type               subtype                                         
ebikes                  emountain_bikes                      2      2   
                        eroad_bikes                          3      3   
                        eutility_bikes                       1      1   
gravel_cyclocross_bikes complete_gravel_cyclocross_bikes    47     47   
mountain_bikes          downhill_bikes                       3      3   
                        enduro_full_suspension_bikes        81     81   
                        hardtail_bikes                      25     25   
                        trail_full_suspension_bikes        146    146   
                        xc_full_suspension_bikes            37     37   
road_bikes              complete_road_bikes                 54     54   
triathlon_tt_bikes      complete_triathlon_tt_bikes          4      4   

                                                          description  price  \
bike_type               subtype                                                
ebikes                  emountain_bikes                             2      2   
                        eroad_bikes                                 3      3   
                        eutility_bikes                              1      1   
gravel_cyclocross_bikes complete_gravel_cyclocross_bikes           47     47   
mountain_bikes          downhill_bikes                              3      3   
                        enduro_full_suspension_bikes               81     81   
                        hardtail_bikes                             25     25   
                        trail_full_suspension_bikes               146    146   
                        xc_full_suspension_bikes                   37     37   
road_bikes              complete_road_bikes                        54     54   
triathlon_tt_bikes      complete_triathlon_tt_bikes                 4      4   

                                                          msrp  tires  \
bike_type               subtype                                         
ebikes                  emountain_bikes                      2      2   
                        eroad_bikes                          3      3   
                        eutility_bikes                       1      1   
gravel_cyclocross_bikes complete_gravel_cyclocross_bikes    47     47   
mountain_bikes          downhill_bikes                       3      3   
                        enduro_full_suspension_bikes        81     56   
                        hardtail_bikes                      25     16   
                        trail_full_suspension_bikes        146    117   
                        xc_full_suspension_bikes            37     28   
road_bikes              complete_road_bikes                 54     53   
triathlon_tt_bikes      complete_triathlon_tt_bikes          4      4   

                                                          bar_tape  \
bike_type               subtype 